In [1]:
# Activate project
using Pkg
Pkg.activate("/Users/alessiogiorlandino/Desktop/SAM/Project.toml")
Base.active_project()
# Resolve dependencies
Pkg.resolve()

  Activating project at `~/Desktop/SAM`


  No Changes to `~/Desktop/SAM/Project.toml`
  No Changes to `~/Desktop/SAM/Manifest.toml`


In [4]:
using EllipsisNotation: (..)
using FillArrays: Falses
import Makie, CairoMakie
using Makie: @colorant_str
import SamApp
using LinearAlgebra
using Statistics

using CSV
using DataFrames

using BioSequences: LongRNA, @rna_str
using Statistics: mean, std, cor

In [6]:
MSA_df=SamApp.rf00162_hits_taxonomy();


In [20]:
MSA_df[!,:domain] = domain
MSA_df[!,:phylum] = phyla


6112-element Vector{String}:
 "Bacillota"
 "Bacillota"
 "Bacillota"
 "Bacillota"
 "Bacillota"
 "Bacillota"
 "Bacillota"
 "Bacillota"
 "Bacillota"
 "Bacillota"
 ⋮
 "Chloroflexi"
 "Bacteroidota"
 "Bacillota"
 "Bacteroidota"
 "Bacillota"
 "Bacteroidota"
 "Actinomycetota"
 "Proteobacteria"
 "Deinococcus-Thermus"

In [21]:
sp_taxonomy = split.(MSA_df[:,"taxonomy"], "; ")
phylum = fill("other", length(sp_taxonomy))
domain = fill("Bacteria", length(sp_taxonomy))
subphylum = fill("other", length(sp_taxonomy))

for i in 1:length(sp_taxonomy)
    if length(sp_taxonomy[i]) >= 3
        domain[i] = sp_taxonomy[i][1]
        phylum[i] = sp_taxonomy[i][2]
        subphylum[i] = sp_taxonomy[i][2]*";"*sp_taxonomy[i][3]
    elseif length(sp_taxonomy[i]) == 2
        domain[i] = sp_taxonomy[i][1]
        phyla[i] = sp_taxonomy[i][2]
    elseif length(sp_taxonomy[i]) == 1
        domain[i] = sp_taxonomy[i][1]
    end
end

MSA_df[!,:domain] = domain
MSA_df[!,:phylum] = phyla
MSA_df[!,:subphylum] = subphylum



#domain_counts = sort!(combine(groupby(MSA_df, :domain), :domain => length => :count), :count, rev=true)


6112-element Vector{String}:
 "Bacillota;Bacilli"
 "Bacillota;Bacilli"
 "Bacillota;Tissierellia"
 "Bacillota;Clostridia"
 "Bacillota;Bacilli"
 "Bacillota;Bacilli"
 "Bacillota;Bacilli"
 "Bacillota;Bacilli"
 "Bacillota;Bacilli"
 "Bacillota;Bacilli"
 ⋮
 "Chloroflexi;Sphaerobacteridae"
 "Bacteroidota;Cytophagia"
 "Bacillota;Clostridia"
 "Bacteroidota;Flavobacteriia"
 "Bacillota;Clostridia"
 "Bacteroidota;Sphingobacteriia"
 "Actinomycetota;Pseudonocardiales"
 "Proteobacteria;Deltaproteobacteria"
 "Deinococcus-Thermus;Deinococci"

In [22]:
phyla_counts = sort!(combine(groupby(MSA_df, :phylum), :phylum => length => :count), :count, rev=true)



Row,phylum,count
,String,Int64
1,Bacillota,3679
2,Bacteroidota,854
3,Actinomycetota,691
4,Pseudomonadota,187
5,Chloroflexi,142
6,Firmicutes,84
7,Fusobacteria,43
8,Deinococcus-Thermus,38
9,Bacteria,33


In [34]:
subphyla_counts = sort!(combine(groupby(MSA_df, :subphylum), :phylum => length => :count), :count, rev=true)


Row,subphylum,count
,String,Int64
1,Bacillota;Bacilli,2382
2,Bacillota;Clostridia,1013
3,Bacteroidota;Flavobacteriia,361
4,other,319
5,Bacteroidota;Cytophagia,246
6,Actinomycetota;Streptomycetales,203
7,Actinomycetota;Micrococcales,170
8,Bacillota;Negativicutes,157
9,Bacteroidota;Sphingobacteriia,149


# Focus On

1   Bacillota;Bacilli	2382 

2	Bacillota;Clostridia	1013 

3	Bacteroidota;Flavobacteriia	361 


4	Bacteroidota;Cytophagia	246 

5	Actinomycetota;Streptomycetales	203 

6	Actinomycetota;Micrococcales	170 

In [59]:
id_Bacilli = MSA_df.id[MSA_df.subphylum .== "Bacillota;Bacilli"]
id_Clostridia = MSA_df.id[MSA_df.subphylum .== "Bacillota;Clostridia"]
id_Flavobacteriia = MSA_df.id[MSA_df.subphylum .== "Bacteroidota;Flavobacteriia"]
id_Cytophagia= MSA_df.id[MSA_df.subphylum .== "Bacteroidota;Cytophagia"]
id_Streptomycetales = MSA_df.id[MSA_df.subphylum .== "Actinomycetota;Streptomycetales"]
id_Micrococcales = MSA_df.id[MSA_df.subphylum .== "Actinomycetota;Micrococcales"]

subphylum_ids = [id_Bacilli, id_Clostridia, id_Flavobacteriia, id_Cytophagia, id_Streptomycetales, id_Micrococcales]

6-element Vector{Vector{String31}}:
 ["CM000733.1/548881-548775", "MLYR01000084.1/7469-7577", "CP007739.1/1700701-1700817", "CP020772.1/4059248-4059357", "JRNX01000191.1/6612-6721", "LFXJ01000010.1/73373-73267", "KI629782.1/602788-602686", "ASQA01000013.1/163414-163521", "CP013659.2/266118-266225", "FQVW01000032.1/3335-3447"  …  "AARI02000249.1/265-361", "CP003056.1/960835-960734", "ACCR02000003.1/83189-83082", "CP003056.1/3328132-3328032", "CP001793.1/5775580-5775452", "ACDZ02000006.1/93274-93376", "CP003056.1/1844232-1844126", "CP001656.1/5519557-5519455", "CP001791.1/1324400-1324509", "CP003056.1/732327-732208"]
 ["LOED01000010.1/15598-15489", "LWDV01000009.1/191001-190896", "JPMD01000017.1/36678-36561", "CP019698.1/2781434-2781540", "LKET01000014.1/116462-116348", "CP006763.1/1556701-1556601", "FRAR01000011.1/111568-111675", "LGGF01000011.1/33512-33621", "LOED01000008.1/4049-3928", "LGFZ01000070.1/5106-5215"  …  "ACIP02000004.1/237213-237114", "AAXA02000011.1/106110-106208", "AAVP0

In [91]:
using StatsBase: sample
N_samples = 150
train_id = []
for id_collection in subphylum_ids
    append!(train_id, sample(id_collection, N_samples, replace=false))
end

train_data = filter(row -> row["id"] in train_id, MSA_df)

Row,id,sequence,organism,taxonomy,domain,phylum,subphylum
,String31,String,String,String,String,String,String
1,JRNX01000191.1/6612-6721,UUCUUAUCAAGAGAGGCAGAGGGACUGGCCCUGUGAAGCCUCAGCAACCACUGGUAUAUCCAGUAAGGUGCUACGUCCAGCAAGUUGUAACAACUUGGAAGAUGAGAA,Halobacillus sp. BBL2006,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Halobacillus,Bacteria,Bacillota,Bacillota;Bacilli
2,FQVW01000032.1/3335-3447,CUCUUAUCCAGAGUGGUGGAGGGAAAGGCCCUACGAAACCACGGCAACCUACAAACAUUUUGGAAAGGUGCCAAAUCCUGCGAAGCGUUAUGCUUUGAAAGAUGAGAG,Ornithinibacillus halophilus,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Ornithinibacillus,Bacteria,Bacillota,Bacillota;Bacilli
3,CP019698.1/2781434-2781540,UUCUUAUCCAGAGUGGCUGAGGGACUGGCCCUGCGAAGCC-CGGCAACCUUCUUUACAUUAAGAAAGGUGCUAAUUCCUGCAGAAGGUAAUCUUCUGAAAGAUAAGAA,Desulforamulus ferrireducens,Bacteria; Bacillota; Clostridia; Eubacteriales; Peptococcaceae; Desulforamulus,Bacteria,Bacillota,Bacillota;Clostridia
4,CP001983.1/3652588-3652482,UCCUUAUCAAGAGAGGUGGAGGGACUGGCCCUGUGAAACC-CGGCAACCGCUGUCUAUGACAGAAUGGUGCCAAAUCCUUAAGAACAGUAUGUUCUUGAAGAUGAGGA,Priestia megaterium QM B1551,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Priestia,Bacteria,Bacillota,Bacillota;Bacilli
5,MSRF01000212.1/5389-5499,CUCUUAUCAAGAGUAGCUGAGGGACUGGCCCAAUGAAGCU-CGGCAACCUCUAUUAAAAAUAGAAAGGUGCUAAAUCCUGCAAAACAUGAAGUUUUGAAAGAUAAGAG,Bacillus sp. VT-16-64,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Bacillus,Bacteria,Bacillota,Bacillota;Bacilli
6,LGGF01000011.1/33512-33621,CUCUUAUCCAGAGAGGUGGAGGGACUGGCCCGAUGAAACC-CGGCAACCGCCUUAAUUAAAGGAAUGGUGCCAAUUCCUGCAGAGGAGAAUCUUCUGACAGAUAAGAG,Desulfotomaculum sp. 46_80,Bacteria; Bacillota; Clostridia; Eubacteriales; Desulfotomaculaceae; Desulfotomaculum,Bacteria,Bacillota,Bacillota;Clostridia
7,CP015438.1/1142271-1142377,UUCUUAUCAAGAGAAGCGGAGGGACUGGCCCUGUGAAGCUUCAGCAACCGCCGGUA---UCGGCCAGGUGCUAAAUCCAGCGAAUUGGAUUAAUUCGGAAGAUAAGAA,Anoxybacillus amylolyticus,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Anoxybacillus,Bacteria,Bacillota,Bacillota;Bacilli
8,MLYR01000039.1/10199-10087,CUCUUAUCCAGAGCGGCGGAGGGAUAGGCCCAAUGAAGCC-CGGCAACCUUCAAGGUACUUGAAAAGGUGCUAAUUCCUGCAGGUUAUAUUAACCUGAAAGAUAAGAG,Bacillus sp. MUM 116,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Bacillus,Bacteria,Bacillota,Bacillota;Bacilli
9,CCAX010000001.1/1527012-1526900,CUCUUAUCUUGAGCGGUGGAGGGACUGGCCCUGUGAAACCGCGGCAACCUUCAACUAUGUUGA-AAGGUGCCAAUUCCUGCAAAGUGAA-AACUUUGACAGAUGAGAG,Oceanobacillus picturae,Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Oceanobacillus,Bacteria,Bacillota,Bacillota;Bacilli


In [30]:
for j in 1:nrow(MSA_df)
    if phyla_counts.count[findall(phyla_counts.phylum.== MSA_df[j, :phylum])][1] < 160 
        MSA_df[!, :phylum][j] = "other"
    end
end

label_file = open("labels.txt","w+")

for label in MSA_df.phylum
    println(label_file, label)
end
close(label_file)



BoundsError: BoundsError: attempt to access 0-element Vector{Int64} at index [1]

In [28]:
subphyla_counts = sort!(combine(groupby(MSA_df, :subphylum), :phylum => length => :count), :count, rev=true)



Row,subphylum,count
,String,Int64
1,Bacillota;Bacilli,2382
2,Bacillota;Clostridia,1013
3,Bacteroidota;Flavobacteriia,361
4,other,319
5,Bacteroidota;Cytophagia,246
6,Actinomycetota;Streptomycetales,203
7,Actinomycetota;Micrococcales,170
8,Bacillota;Negativicutes,157
9,Bacteroidota;Sphingobacteriia,149
